In [1]:
from pymongo import MongoClient
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import MongoDBAtlasVectorSearch
from langchain_community.document_loaders import DirectoryLoader
from langchain_openai import OpenAI
from langchain.chains import RetrievalQA
import gradio as gr
from gradio.themes.base import Base
import key_param

/Users/timhulshof/Documents/RAG/venv/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/timhulshof/Documents/RAG/venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
client = MongoClient(key_param.MONGO_URI)
dbName = "IMD"
collectionName = "collection_of_IMD_text_blobs"
collection = client[dbName][collectionName]

In [3]:
import os
import shutil

source_dir = "../agora/IMDs/"  # The directory containing the subfolders with text files
destination_dir = "./IMD_text_files/"  # The folder you created to store the copies

# Ensure the destination directory exists
os.makedirs(destination_dir, exist_ok=True)

# Iterate through each subfolder within the source directory
for subfolder in os.listdir(source_dir):
    subfolder_path = os.path.join(source_dir, subfolder)

    # Check if the item is actually a subfolder
    if os.path.isdir(subfolder_path):
        # Iterate over the files within the subfolder
        for file in os.listdir(subfolder_path):
            file_path = os.path.join(subfolder_path, file)

            # Check if it's a text file
            if file.endswith(".txt"):
                shutil.copy(file_path, destination_dir)  # Copy the file


In [4]:
loader = DirectoryLoader('./IMD_text_files', glob='./*.txt', show_progress=True)
data = loader.load()

embeddings = OpenAIEmbeddings(openai_api_key=key_param.openai_api_key)

  0%|          | 0/109 [00:00<?, ?it/s]

100%|██████████| 109/109 [00:04<00:00, 24.06it/s]


In [6]:
vectorStore = MongoDBAtlasVectorSearch.from_documents(data, embeddings, collection=collection)